In [ ]:
import pandas as pd
import requests

import geopandas as gpd
import pandas as pd
from rasterstats import zonal_stats
import glob
import os
import rasterio
import numpy as np

In [71]:
with rasterio.open("/Users/cherryleheu/Documents/HCDP/Data/climo/monthly/rainfall/monthly_rainfall_clim_statewide_1991-2020_8.tif") as src:
    rf = src.read(1)
    nodata_climo = src.nodata

masked_climo = np.ma.masked_equal(rf, nodata_climo)
masked_climo_avg = masked_climo.mean()/25.4

In [73]:
with rasterio.open("/Users/cherryleheu/Documents/HCDP/Data/monthly/rainfall/rainfall_2025_08.tif") as src:
    rf = src.read(1)
    nodata = src.nodata

In [35]:
raster_folder = "/Users/cherryleheu/Documents/HCDP/Data/monthly/rainfall/"
climatology_file = "/Users/cherryleheu/Documents/HCDP/Data/climo/monthly/rainfall/monthly_rainfall_clim_statewide_1991-2020_8.tif"
records = []

# Process monthly rasters
for tif in sorted(glob.glob(os.path.join(raster_folder, "rainfall*_08.tif"))):
    # Extract date from filename rainfall_1990_01.tif → 1990-01
    parts = os.path.basename(tif).replace(".tif", "").split("_")
    year, month = parts[1], parts[2]
    date = f"{year}-{month}"
    with rasterio.open(tif) as src:
        rf = src.read(1)
        nodata = src.nodata

    masked_data = np.ma.masked_equal(rf, nodata)
    masked_data_avg = masked_data.mean()/25.4

    records.append({
        "date": date,
        "mean": masked_data_avg,
        "anomaly": masked_data_avg - masked_climo_avg
    })

# Convert to DataFrame
df = pd.DataFrame(records)

# Rank anomalies (1 = driest, N = wettest)
df["rank"] = df["anomaly"].rank(method="min")

# Save output
# df.to_csv("../public/statewide_rainfall_anomaly_rank.csv", index=False)

print(df)

       date       mean    anomaly  rank
0   1990-08   3.009804  -2.151147  10.0
1   1991-08   7.887854   2.726903  32.0
2   1992-08   5.060304  -0.100647  26.0
3   1993-08   4.679400  -0.481551  24.0
4   1994-08   6.771792   1.610841  30.0
5   1995-08   5.432207   0.271256  27.0
6   1996-08   2.624407  -2.536544   5.0
7   1997-08   3.107671  -2.053280  11.0
8   1998-08   4.460803  -0.700148  21.0
9   1999-08   3.468746  -1.692205  14.0
10  2000-08   5.862320   0.701369  29.0
11  2001-08   3.610164  -1.550787  16.0
12  2002-08   4.580658  -0.580293  22.0
13  2003-08   4.305691  -0.855260  20.0
14  2004-08   4.679465  -0.481486  25.0
15  2005-08   3.886228  -1.274723  18.0
16  2006-08   3.416718  -1.744233  13.0
17  2007-08   3.810366  -1.350585  17.0
18  2008-08   2.819953  -2.340998   7.0
19  2009-08   3.975136  -1.185815  19.0
20  2010-08   2.108373  -3.052578   2.0
21  2011-08   2.873382  -2.287569   8.0
22  2012-08   3.243183  -1.917768  12.0
23  2013-08   2.983813  -2.177138   9.0


In [90]:
shapefile = "../public/hawaii_climate_divisions.shp"  # replace with your divisions shapefile
raster_folder = "/Users/cherryleheu/Documents/HCDP/Data/monthly/rainfall/"
climatology_file = "/Users/cherryleheu/Documents/HCDP/Data/climo/monthly/rainfall/monthly_rainfall_clim_statewide_1991-2020_8.tif"

# Load shapefile
gdf = gpd.read_file(shapefile)

# Climatology mean per division
climo_stats = zonal_stats(
    vectors=gdf,
    raster=climatology_file,
    stats=["mean"],
    nodata=nodata_climo
)
gdf["climo_mean"] = [c["mean"]/25.4 for c in climo_stats]  # mm → in

# Collect all results
all_records = []

# Loop over August rasters
for tif in sorted(glob.glob(os.path.join(raster_folder, "rainfall*_08.tif"))):
    parts = os.path.basename(tif).replace(".tif", "").split("_")
    year, month = parts[1], parts[2]
    date = f"{year}-{month}"

    if year == "1990":
        continue
    stats = zonal_stats(
        vectors=gdf,
        raster=tif,
        stats=["mean"],
        nodata=nodata
    )

    for idx, division in gdf.iterrows():
        mean_val = stats[idx]["mean"]/25.4 if stats[idx]["mean"] is not None else np.nan
        climo_mean = division["climo_mean"]
        anomaly = mean_val - climo_mean if mean_val is not None else np.nan

        all_records.append({
            "division": division["name"],  # adjust to shapefile column
            "date": date,
            "mean": mean_val,
            "anomaly": anomaly
        })

# Convert to DataFrame
df = pd.DataFrame(all_records)

# Rank anomalies per division
df["rank"] = df.groupby("division")["anomaly"].rank(method="min")

division_dfs = {div: subdf.reset_index(drop=True) for div, subdf in df.groupby("division")}

aug2025_df = df[df["date"] == "2025-08"].reset_index(drop=True)

print("Division-specific DataFrames created:", list(division_dfs.keys()))
print("August 2025 DataFrame:")
print(aug2025_df)

Division-specific DataFrames created: ['Hawaiʻi Mauka', 'Hilo', 'Kaʻu', 'Kona', 'Koʻolau', 'Leeward Kauaʻi', 'Leeward Kohala', 'Leeward Maui Nui', 'Waianae', 'Windward Kauaʻi', 'Windward Kohala', 'Windward Maui Nui']
August 2025 DataFrame:
             division     date      mean   anomaly  rank
0      Leeward Kauaʻi  2025-08  0.451942 -1.711460   1.0
1     Windward Kauaʻi  2025-08  2.991385 -4.932987   2.0
2             Waianae  2025-08  0.291740 -1.276930   1.0
3             Koʻolau  2025-08  1.696667 -5.104172   1.0
4    Leeward Maui Nui  2025-08  0.377159 -0.734619   6.0
5   Windward Maui Nui  2025-08  1.810440 -5.795241   1.0
6      Leeward Kohala  2025-08  0.644451 -0.613472  10.0
7     Windward Kohala  2025-08  2.300009 -4.084889   3.0
8                Kona  2025-08  1.972189 -0.993316   8.0
9       Hawaiʻi Mauka  2025-08  0.946217 -1.037262  11.0
10               Kaʻu  2025-08  2.216744 -1.850943  15.0
11               Hilo  2025-08  4.698646 -8.110893   3.0


In [92]:
shapefile = "../public/hawaii_climate_divisions.shp"  # replace with your divisions shapefile
raster_folder = "/Users/cherryleheu/Documents/HCDP/Data/monthly/rainfall/"
climatology_file = "/Users/cherryleheu/Documents/HCDP/Data/climo/monthly/rainfall/monthly_rainfall_clim_statewide_1991-2020_8.tif"

# Load shapefile
gdf = gpd.read_file(shapefile)

# Climatology mean per division
climo_stats = zonal_stats(
    vectors=gdf,
    raster=climatology_file,
    stats=["sum"],
    nodata=nodata_climo
)
gdf["climo_mean"] = [c["sum"]/25.4 for c in climo_stats]  # mm → in

# Collect all results
all_records = []

# Loop over August rasters
for tif in sorted(glob.glob(os.path.join(raster_folder, "rainfall*_08.tif"))):
    parts = os.path.basename(tif).replace(".tif", "").split("_")
    year, month = parts[1], parts[2]
    date = f"{year}-{month}"

    if year == "1990":
        continue
    stats = zonal_stats(
        vectors=gdf,
        raster=tif,
        stats=["sum"],
        nodata=nodata
    )

    for idx, division in gdf.iterrows():
        mean_val = stats[idx]["sum"]/25.4 if stats[idx]["sum"] is not None else np.nan
        climo_mean = division["climo_mean"]
        anomaly = mean_val - climo_mean if mean_val is not None else np.nan

        all_records.append({
            "division": division["name"],  # adjust to shapefile column
            "date": date,
            "mean": mean_val,
            "anomaly": anomaly
        })

# Convert to DataFrame
df = pd.DataFrame(all_records)

# Rank anomalies per division
df["rank"] = df.groupby("division")["anomaly"].rank(method="min")

division_dfs = {div: subdf.reset_index(drop=True) for div, subdf in df.groupby("division")}

aug2025_df = df[df["date"] == "2025-08"].reset_index(drop=True)

print("Division-specific DataFrames created:", list(division_dfs.keys()))
print("August 2025 DataFrame:")
print(aug2025_df)

Division-specific DataFrames created: ['Hawaiʻi Mauka', 'Hilo', 'Kaʻu', 'Kona', 'Koʻolau', 'Leeward Kauaʻi', 'Leeward Kohala', 'Leeward Maui Nui', 'Waianae', 'Windward Kauaʻi', 'Windward Kohala', 'Windward Maui Nui']
August 2025 DataFrame:
             division     date           mean        anomaly  rank
0      Leeward Kauaʻi  2025-08    4521.681225  -17123.153912   1.0
1     Windward Kauaʻi  2025-08   44434.035433  -73274.586614   2.0
2             Waianae  2025-08    4505.631152  -19720.909203   1.0
3             Koʻolau  2025-08   19146.887303  -57600.583169   1.0
4    Leeward Maui Nui  2025-08   12717.790354  -24771.348425   6.0
5   Windward Maui Nui  2025-08   30239.773622  -96797.903543   1.0
6      Leeward Kohala  2025-08   12988.265256  -12363.914862  10.0
7     Windward Kohala  2025-08   48012.691929  -85272.052165   3.0
8                Kona  2025-08   44271.707677  -22297.952756   8.0
9       Hawaiʻi Mauka  2025-08   28947.603346  -31732.947835  11.0
10               Kaʻu  